# Chapter-3: Create and train a ML segmentation model using AWS SageMaker

The objectives you complete during the course of this chapter introduce you to the process of implementing the SageMaker model training tool. You will engage in this process by completing the objectives listed below.

### AWS SageMaker:
Amazon SageMaker helps data scientists and developers prepare, build, train, and deploy high-quality machine learning (ML) models quickly by bringing together a broad set of capabilities purpose-built for ML.

Simply put, It is a set of cloud-based (specifically, AWS) apps that focus on labeling, training, testing and deploying models.

## How it works in context of our HLD problem:

### Prepare

SageMaker provides in-house labelling tools and data wrangling tools for some common ML workflows. Owing to the spatiotemporal nature of the HLD dataset, we will be skipping this functionality of SageMaker and use the ImageLabeler tool, built by IMPACT, for identifying features and labeling them. This workflow has been covered in Chapter-0 and Chapter-1 of this workshop.

### Build, train, and tune

SageMaker provides access to cloud-hosted Jupyter notebooks along with pre-built ML models. For our purposes, the model we are using for this demo is the UNet segmentation model (https://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/). The architecture is a stack of convolutions followed by de-convolutions. This model assigns a class label to each pixel of the input and gives an output matching the size of the input. The resulting output, once trained with high-latitude dust (HLD) masks, will segment any given image into HLD and non-HLD pixels. We will be covering the process in this chapter (Chapter-3).

### Deploy and manage

SageMaker provides endpoints to infer from the trained models. This functionality will be showcased in Chapter-4.


In [ ]:
!pip install -r src/requirements.txt

## Required Imports

In [ ]:
import boto3
import fiona

import math
import numpy as np
import os
import random
import rasterio.features
import re
import requests
import shutil

from datetime import datetime
from glob import glob
from io import BytesIO
from IPython.display import Image as Display
from PIL import Image


## From Chapter-2: setup access, download, and process data into ML-ready format.

In [ ]:
ACCOUNT_NUMBER = "350996086543"
ROLE_NAME = "notebookAccessRole"
ROLE_ARN = f"arn:aws:iam::{ACCOUNT_NUMBER}:role/{ROLE_NAME}"
SOURCE_BUCKET = "impact-datashare"
BUCKET_NAME = f"{ACCOUNT_NUMBER}-model-bucket"
DESTINATION_BUCKET = f"s3://{BUCKET_NAME}"

## Initialize a SageMaker session to upload data

In [ ]:
import sagemaker
sagemaker_session = sagemaker.Session()
train_images = sagemaker_session.upload_data(path='data/train')
val_images = sagemaker_session.upload_data(path='data/val')
test_images = sagemaker_session.upload_data(path='data/test')

## Use the TensorFlow wrapper provided by SageMaker to train a UNet model

The SageMaker Python SDK TensorFlow estimators and models and the SageMaker open-source TensorFlow containers make writing a TensorFlow script and running it in SageMaker easier.

We define a SageMaker instance using `sagemaker.tensorflow.Tensorflow` class. 

- `entry_point` parameter should point to the underlying TensorFlow model implementation.
- `source_dir` points to the folder that contains the `entry_point`.
- `role` should be given the appropriate notebook access role we created in Chaper-2.
- `instance_count`, `instance_type` defines the number of instances and the type of instance of the EC2 instance that will be used for compute.
- `output_path` - dictates where the output files from training the model will reside.
- `image_uri` should point to the appropriate TensorFlow ECR container image.


In [ ]:
from sagemaker.tensorflow import TensorFlow
keras_metric_definition = [
    {"Name": "train:loss", "Regex": ".*loss: ([0-9\\.]+) - accuracy: [0-9\\.]+.*"},
    {"Name": "train:accuracy", "Regex": ".*loss: [0-9\\.]+ - accuracy: ([0-9\\.]+).*"},
    {
        "Name": "validation:accuracy",
        "Regex": ".*step - loss: [0-9\\.]+ - accuracy: [0-9\\.]+ - val_loss: [0-9\\.]+ - val_accuracy: ([0-9\\.]+).*",
    },
    {
        "Name": "validation:loss",
        "Regex": ".*step - loss: [0-9\\.]+ - accuracy: [0-9\\.]+ - val_loss: ([0-9\\.]+) - val_accuracy: [0-9\\.]+.*",
    },
    {
        "Name": "sec/steps",
        "Regex": ".* (\d+)[mu]s/step - loss: [0-9\\.]+ - accuracy: [0-9\\.]+ - val_loss: [0-9\\.]+ - val_accuracy: [0-9\\.]+",
    },
]

LOG_FOLDER = "tensorboard_logs"

TENSORBOARD_LOGS_PATH = f"s3://{BUCKET_NAME}/{LOG_FOLDER}/"

estimator = TensorFlow(
    entry_point='hld_sagemaker_demo.py',
    source_dir="/home/ec2-user/SageMaker/workshop_notebooks/chapter-3/src",
    role=ROLE_NAME,
    instance_count=1,
    instance_type='ml.p3.2xlarge',
    py_version='py3',
    hyperparameters={"log_dir": TENSORFLOW_LOGS_PATH},
    output_path=DESTINATION_BUCKET,
    image_uri='763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.4.1-gpu-py37-cu110-ubuntu18.04',
    metric_definition=keras_metric_definition,
    distribution={
        'parameter_server': {'enabled': True}
    }
)


## Train the model

Training the model is as simple as calling estimator.fit() and providing it appropriate arguments that are expected by `hld_sagemaker_demo.py`, which is the entry point for training the custom model (from the previous step).

In [ ]:
estimator.fit(
    {
        'train': train_images,
        'eval': val_images, 
        'test': test_images
    }
)

## Attach a TensorBoard session

TensorBoard in an interactive model training visualization tool. It lets us monitor the training/validation losses over time and gives an idea of how well the model is training. (Since we are passing the logs to S3, we can interact with the logs directly from our local setup.)
Here's how you'd connect your TensorBoard to your S3 bucket:
1. install TensorBoard version 2.4.1 using `pip install tensorboard==2.4.1`
2. run `AWS_REGION=<the region you selected> tensorboard --logdir s3://<your account number>-model-bucket/tensorboard_logs/`
3. navigate to localhost:6006 to view the model metrics, model graph timeline, and more.


You can also try running it via this very notebook using the following cell. 

In [ ]:
# The corresponding TensorBoard can be accessed using `https://<your-notebook-instance-name>/proxy/6006/` the https and the trailing spaces are very important.
aws_region = sagemaker_session.boto_region_name
!AWS_REGION={aws_region} tensorboard --logdir {TENSORBOARD_LOGS_PATH}

# Note the trained model name in the DESTINATION_BUCKETdashboard

This will be used in the next chapter to deploy and infer from the model.